In [88]:
from utils import *
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder

In [113]:
data = load_data('data/mnist_small.csv', has_header=False)

X_data = data[0]

Y_data = np.where(data[1]>4, 1.0, .0)

n = X_data.shape[1]
m = Y_data.size

In [114]:
run_no=0

In [117]:
X_r, Y_r, X_e, Y_e = train_test_split(X_data, Y_data, .8)

n_units = 200



tf.reset_default_graph()
run_no += 1
with tf.device('/cpu:0'):
# if True:
    X = tf.placeholder(X_data.dtype, (None, n), 'features')
    Y = tf.placeholder(Y_data.dtype, (None, 1), 'label')
    L1 = tf.layers.dense(inputs=X, units=n_units, activation=tf.nn.sigmoid, name='H1')
#     L2 = tf.layers.dense(inputs=L1, units=10, activation=tf.nn.sigmoid, name='H2')
#     regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
#     dropout = tf.layers.dropout(L1, 1)
#     weights = tf.get_variable('o_weights', (n_units, 1), X.dtype, tf.random_uniform_initializer())
#     logits = tf.m atmul(L1, weights)
    BC = tf.layers.dense(inputs=L1, units=1, name='OUT')#, kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1))  
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=BC))
    tf.summary.scalar('loss', loss)
    optimiser = tf.train.AdamOptimizer(learning_rate=.01).minimize(loss)
    writer = tf.summary.FileWriter('data/.graph/run{}/train'.format(run_no), tf.get_default_graph())
    test_writer = tf.summary.FileWriter('data/.graph/run{}/test'.format(run_no))
    merged_smr = tf.summary.merge_all()
    inference = tf.nn.sigmoid(BC)

epochs = 500
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
#         print(loss)
        smr = sess.run(merged_smr, feed_dict={X: X_r, Y: Y_r})
        writer.add_summary(smr, i)
        smr = sess.run(merged_smr, feed_dict={X: X_e, Y: Y_e})
        test_writer.add_summary(smr, i)
        sess.run(optimiser, feed_dict={X: X_r, Y: Y_r})
    prd = sess.run(inference, feed_dict={X: X_e})
    prd_r = sess.run(inference, feed_dict={X: X_r})
    
writer.close()

prd = np.where(prd<.5, 0, 1)
prd_r = np.where(prd_r<.5, 0, 1)

print('ANN f1-score =', f1_score(Y_e, prd))
print('ANN f1-score train =', f1_score(Y_r, prd_r))

clf = KNeighborsClassifier(n_neighbors=3, weights='distance')
clf.fit(X_r, Y_r.astype(int))

k_prd = clf.predict(X_e)

print('KNN f1-score =', f1_score(Y_e.astype(int), k_prd))

ANN f1-score = 0.967741935483871
ANN f1-score train = 0.9868421052631579
KNN f1-score = 0.8666666666666667


/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


In [150]:
data = load_data('data/mnist_large.csv', has_header=False)

X_data = data[0]
Y_data = data[1]

enc = OneHotEncoder()
Y_OH = enc.fit_transform(Y_data).toarray()


n = X_data.shape[1]
m = Y_data.size

X_r, Y_r, X_e, Y_e = train_test_split(X_data, Y_OH, .8)

n_units = 100


tf.reset_default_graph()
run_no += 1
with tf.device('/cpu:0'):
    X = tf.placeholder(X_data.dtype, (None, n), 'features')
    Y = tf.stop_gradient(tf.placeholder(Y_data.dtype, (None, 10), 'label'))
    L1 = tf.layers.dense(inputs=X, units=n_units, activation=tf.nn.sigmoid, name='H1')
#     dropout = tf.layers.dropout(L1, 10)
#     L2 = tf.layers.dense(inputs=L1, units=10, activation=tf.nn.sigmoid, name='H2')
#     regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
#     weights = tf.get_variable('o_weights', (n_units, 1), X.dtype, tf.random_uniform_initializer())
#     logits = tf.m atmul(L1, weights)
    BC = tf.layers.dense(inputs=L1, units=10, name='OUT')#, kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=1))  
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=BC))
    tf.summary.scalar('loss', loss)
    optimiser = tf.train.AdamOptimizer(learning_rate=.01).minimize(loss)
    writer = tf.summary.FileWriter('data/.graph/run{}/train'.format(run_no), tf.get_default_graph())
    test_writer = tf.summary.FileWriter('data/.graph/run{}/test'.format(run_no))
    merged_smr = tf.summary.merge_all()
    inference = tf.nn.softmax(BC)

epochs = 500
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs):
#         print(loss)
        smr = sess.run(merged_smr, feed_dict={X: X_r, Y: Y_r})
        writer.add_summary(smr, i)
        smr = sess.run(merged_smr, feed_dict={X: X_e, Y: Y_e})
        test_writer.add_summary(smr, i)
        sess.run(optimiser, feed_dict={X: X_r, Y: Y_r})
    prd = sess.run(inference, feed_dict={X: X_e, Y: Y_e})
    prd_r = sess.run(inference, feed_dict={X: X_r, Y: Y_r})
writer.close()

In [151]:
prd = np.round(prd)
prd_r = np.round(prd_r)

print('ANN f1-score =', f1_score(Y_e, prd, average='weighted'))
print('ANN f1-score train =', f1_score(Y_r, prd_r, average='weighted'))

clf = KNeighborsClassifier(n_neighbors=3, weights='distance')
clf.fit(X_r, Y_r.astype(int))

k_prd = clf.predict(X_e)

print('KNN f1-score =', f1_score(Y_e.astype(int), k_prd, average='weighted'))

ANN f1-score = 0.8745475045612437
ANN f1-score train = 0.9995813953488373
KNN f1-score = 0.8881751568367922


## Judging by test and train loss function charts, the ANN overfits data. This issue may be overcome by adding data to training set or regularisation. Adding hidden layers only leads to  even worse overfitting.
## With these conditions, KNN performs better even without significant tuning.